<a href="https://colab.research.google.com/github/Sachidananda-17/BreakoutAi/blob/main/BreakoutAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# List of popular cryptocurrencies with their CoinGecko IDs and names
popular_cryptocurrencies = {
    'bitcoin': 'Bitcoin',
    'ethereum': 'Ethereum',
    'dogecoin': 'Dogecoin',
    'ripple': 'Ripple',
    'litecoin': 'Litecoin',
    'cardano': 'Cardano',
    'polkadot': 'Polkadot',
    'binancecoin': 'Binance Coin',
    'solana': 'Solana',
    'chainlink': 'Chainlink',
    'stellar': 'Stellar',
    'uniswap': 'Uniswap',
    'bitcoin-cash': 'Bitcoin Cash',
    'tether': 'Tether'
}

# Step 1: Fetch historical data using CoinGecko API
def fetch_crypto_data(crypto_id: str, vs_currency: str, days: int) -> pd.DataFrame:
    base_url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {
        "vs_currency": vs_currency,
        "days": days,  # Retrieve up to the maximum range of 365 days
        "interval": "daily"
    }

    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code != 200:
        print("Error fetching data:", response.json().get("error", "Unknown error"))
        return pd.DataFrame()  # Return empty DataFrame if API call fails

    data = response.json()

    # Process data into DataFrame
    records = []
    for i in range(len(data['prices'])):
        date = datetime.fromtimestamp(data['prices'][i][0] / 1000).strftime('%Y-%m-%d')
        price = data['prices'][i][1]
        records.append({
            "Date": date,
            "Open": price,
            "High": price,
            "Low": price,
            "Close": price
        })

    df = pd.DataFrame(records)
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' column is datetime type

    return df

# Step 2: Calculate metrics
def calculate_metrics(data: pd.DataFrame, variable1: int, variable2: int) -> pd.DataFrame:
    data['High_Last_{}_Days'.format(variable1)] = data['High'].rolling(window=variable1).max()
    data['Low_Last_{}_Days'.format(variable1)] = data['Low'].rolling(window=variable1).min()

    # Calculate future high and low prices
    data['High_Next_{}_Days'.format(variable2)] = data['High'].shift(-variable2).rolling(window=variable2).max()
    data['Low_Next_{}_Days'.format(variable2)] = data['Low'].shift(-variable2).rolling(window=variable2).min()

    return data

# Step 3: Data preprocessing for LSTM
def prepare_data_for_lstm(data: pd.DataFrame, look_back: int):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['High_Last_7_Days', 'Low_Last_7_Days', 'Close']].dropna())

    X, y_high, y_low = [], [], []
    for i in range(look_back, len(scaled_data) - 5):
        X.append(scaled_data[i - look_back:i])
        y_high.append(scaled_data[i, 0])  # Future high
        y_low.append(scaled_data[i, 1])   # Future low

    X, y_high, y_low = np.array(X), np.array(y_high), np.array(y_low)
    return X, y_high, y_low, scaler

# Step 4: Build and train LSTM model
def build_and_train_lstm(X, y):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

    return model

# Step 5: Predict future prices
def predict_future_prices(model_high, model_low, X, scaler):
    predicted_high = model_high.predict(X)
    predicted_low = model_low.predict(X)

    # Reverse scaling
    predicted_high = scaler.inverse_transform(np.concatenate([predicted_high, np.zeros((predicted_high.shape[0], 2))], axis=1))[:, 0]
    predicted_low = scaler.inverse_transform(np.concatenate([predicted_low, np.zeros((predicted_low.shape[0], 2))], axis=1))[:, 0]

    return predicted_high, predicted_low

# Main Code Execution
if __name__ == "__main__":
    # Display the list of popular cryptocurrencies
    print("Popular Cryptocurrencies:")
    for crypto_id, crypto_name in popular_cryptocurrencies.items():
        print(f"{crypto_id}: {crypto_name}")

    # Ask user to select a cryptocurrency ID
    crypto_id = input("\nChoose a cryptocurrency ID from the list above (e.g., 'bitcoin'): ").strip().lower()

    # Validate cryptocurrency selection
    if crypto_id not in popular_cryptocurrencies:
        print("Invalid cryptocurrency selected. Exiting.")
        exit()

    vs_currency = "usd"  # Set comparison currency to USD
    days = 365  # Fetch up to 365 days of historical data (CoinGecko limit)

    # Fetch data from CoinGecko API based on user input
    crypto_data = fetch_crypto_data(crypto_id, vs_currency, days)

    # Check if data was successfully fetched
    if crypto_data.empty:
        print("No data fetched. Please check the API or crypto_id input.")
    else:
        # Step 2: Calculate metrics
        crypto_data_with_metrics = calculate_metrics(crypto_data, variable1=7, variable2=5)

        # Drop NaNs from metric calculations
        crypto_data_with_metrics.dropna(inplace=True)

        # Step 3: Prepare data for LSTM
        X, y_high, y_low, scaler = prepare_data_for_lstm(crypto_data_with_metrics, look_back=30)

        # Step 4: Build and train LSTM models for high and low prices
        model_high = build_and_train_lstm(X, y_high)
        model_low = build_and_train_lstm(X, y_low)

        # Step 5: Predict future prices using the last segment of data
        predicted_high, predicted_low = predict_future_prices(model_high, model_low, X[-1:], scaler)
        print(f"\nPredicted Future High Price ({popular_cryptocurrencies[crypto_id]} in {vs_currency.upper()}): {predicted_high[-1]:.2f}")
        print(f"Predicted Future Low Price ({popular_cryptocurrencies[crypto_id]} in {vs_currency.upper()}): {predicted_low[-1]:.2f}")


Popular Cryptocurrencies:
bitcoin: Bitcoin
ethereum: Ethereum
dogecoin: Dogecoin
ripple: Ripple
litecoin: Litecoin
cardano: Cardano
polkadot: Polkadot
binancecoin: Binance Coin
solana: Solana
chainlink: Chainlink
stellar: Stellar
uniswap: Uniswap
bitcoin-cash: Bitcoin Cash
tether: Tether

Choose a cryptocurrency ID from the list above (e.g., 'bitcoin'): ethereum
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 0.2538 - val_loss: 0.0432
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0472 - val_loss: 0.0042
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0268 - val_loss: 0.0054
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0203 - val_loss: 0.0162
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0191 - val_loss: 0.0057
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0166 - val_loss: 0.0045
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0136 - val_loss: 0.0089
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0141 - val_loss: 0.0034
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0129 - val_loss: 0.0037
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0120 - val_loss: 0.0036
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0105 - val_loss: 0.0031
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0082 - val_loss: 0.0040
Epoch 13/50
